In [1]:
import os
from pathlib import Path
import importlib
import numpy as np
import nibabel as nib

os.chdir("/home/srs-9/Projects/ms_mri/monai")

from scripts import data_file_manager as datafm

In [115]:
importlib.reload(datafm)

<module 'scripts.data_file_manager' from '/home/srs-9/Projects/ms_mri/monai/scripts/data_file_manager.py'>

In [8]:
base_path = Path("/mnt/f/Data/ms_mri/Brats_3d_seg/Task01_BrainTumour")
imagesTr = base_path / "imagesTr"
image_files = os.scandir(imagesTr)
image_paths = [file.path for file in image_files if "._" not in file.name]

In [9]:
img = nib.load(image_paths[0])

a = np.array(img.dataobj)

In [117]:
base_path = Path("/mnt/f/Data/ms_mri/lesjak_2017")
data_dir = base_path / "data"

lesjak_datafm = datafm.LesjakData(data_dir)
subjects = lesjak_datafm.subjects['name']
labels = lesjak_datafm.assign_train_test_data(subjects, 0.2)

TypeError: list indices must be integers or slices, not str

In [108]:
data_dir

PosixPath('/mnt/c/Users/srs-9/OneDrive - UMass Chan Medical School/Projects/MS MRI/lesjak_2017/data')

In [98]:
test = tuple()
if not test:
    print("uninitialzed")

uninitialzed


In [ ]:
import random
fraction_ts = 0.2

n_subj = len(subjects)
n_ts = int(n_subj * fraction_ts)

random.shuffle(subjects)
subj_ts = [(subj, "ts") for subj in subjects[:n_ts]]
subj_tr = [(subj, "tr") for subj in subjects[n_ts:]]

subj_ts + subj_tr